# Reduction

Delta Debugging is a method to reduce failure inducing inputs to their smallest
required size that still induces the same failure. It was first formally introduced
in the paper _Simplifying and Isolating Failure-Inducing Input_ by _Zeller_ and _Hildebrandt_.

The idea of delta debugging is fairly simple. We start by partitioning the given input
string, starting with two partitions – which have a given partition length.
Then, we check if any of these parts can be removed without removing the observed
failure. If any of these can be removed, we remove all such parts of the given length.
Once no such parts of the given length can be removed, we reduce the partition length
by two, and do the same process again. This obtains us the 1-minimal failure causing
string where removal of even a single character will remove the observed failure.

## Delta Debugging

### remove_check_each_fragment()

Given a partition length, we want to split the string into
that many partitions, remove each partition one at a time from the
string, and check if for any of them, the `causal()` succeeds. If it
succeeds for any, then we can skip that section of the string.

**Note.** A more detailed treatment of delta debugging can be found [here](https://rahul.gopinath.org/post/2019/12/03/ddmin/).

In [ ]:
def remove_check_each_fragment(instr, part_len, causal):
    pre = ''
    for i in range(0, len(instr), part_len):
        stitched =  pre + instr[i+part_len:]
        if not causal(stitched):
             pre = pre + instr[i:i+part_len]
    return pre

There is a reason this function is split from the main function unlike in the
original implementation of `ddmin`. The function `remove_check_each_fragment`
obeys the contract that any string returned by it obeys the contract represented
by the `causal` function. This means that any test case that is produced by
`remove_check_each_fragment` will reproduce the specified behavior, and can be
used for other computations. For example, one may use it for evaluating test
reduction slippage, or for finding other reductions.


### ddmin()

The main function. We start by the smallest number of partitions -- 2.
Then, we check by removing each fragment for success. If removing one
fragment succeeds, we change the current string to the string without that
fragment. So, we remove all fragments that can be removed in that partition
size.
If none of the fragments could be removed, then we reduce the partition length
by half.
If the partition cannot be halved again (i.e, the last partition length was
one) or the string has become empty, we stop the iteration.

In [ ]:
def ddmin(cur_str, causal_fn):
    part_len = len(cur_str) // 2
    while part_len and cur_str:
        _str = remove_check_each_fragment(cur_str, part_len, causal_fn)
        if _str == cur_str:
            part_len = part_len // 2
        cur_str = _str

    return cur_str

The driver.

In [ ]:
def test(s):
    print("%s %d" % (s, len(s)))
    return set('()') <= set(s)

In [ ]:
if __name__ == '__main__':
    import random, string
    inputstring = ''.join(random.choices(string.digits +
                          string.ascii_letters +
                          string.punctuation, k=1024))
    print(inputstring)

In [ ]:
if __name__ == '__main__':
    assert test(inputstring)
    solution = ddmin(inputstring, test)
    print(solution)

However, pure _ddmin_ is limited when it comes to structured inputs.
One of the problems with the unstructured version of `ddmin()` above is that
it assumes that parts of the inputs can be cut away, while still retaining
validity in that it will actually reach the testing function. This, however,
may not be a reasonable assumption especially in the case of structured
inputs. The problem is that if you have a JSON `[{"a": null}]` that produces
an error because the key value is `null`, `ddmin()` will try to partition it
as `[{"a":` followed by `null}]` neither of which are valid. Further, any
chunk removed from either parts are also likely to be invalid. Hence,
`ddmin()` will not be able to proceed.

# Hierarchical Delta Debugging

The solution here is to go for a variant called *Hierarchical Delta Debugging*
(Misherghi 2006).
The basic idea is to first extract the parse tree (either by parsing the input
using a grammar, or by either relying on a generator to generate the parse
tree along with the input, or by extracting the parse tree in-flight from the
target program if it supports it), and then try and apply `ddmin()` on each
level of the derivation tree. Another notable improvement was invented by
Herfert et al. (Herfert 2017).
In this paper, the authors describe a simple strategy: Try and replace a given
node by one of its child nodes. This works reasonably well for inputs that are
context sensitive such as programs that can trigger bugs in compilers.
A final algorithm is Perses (Sun 2018) which uses the program grammar
directly to avoid creating invalid nodes. In this post, I will explain a simplified
variant of the algorithm of *Perses*. I note that a similar idea was explored by
Pike in 2014 (Pike 2014) albeit for data structures.

The basic idea of Perses is to try and replace a parent node by either an
empty expansion (so that the corresponding string fragment can be deleted)
or by a child node of the same nonterminal. To accomplish the first (that is,
to be able to use empty expansions), Perses first transofrms the given grammar
to a *Perses normal form*. We can however skip that portion if we are careful
and use a grammar where we explicitly use nonterminals that can have an empty
expansion. In this post, I only explain the *perses reduction* algorithm.

**Note.** A more detailed treatment of hierarchical delta debugging can be found [here](https://rahul.gopinath.org/post/2019/12/04/hdd/).

## Predicate
For the delta debug algorithm to work, we need a predicate. However, unlike
*ddmin* which works with simple pass or fail status, we need a bit more
sophistication here. The reason is that in many cases, the newly constructed
strings we generate may also be *invalid*. For example, a particular program
may produce a core dump from the C compiler. Hence, the core dump is the
`pass` for the predicate. On the other hand, deleting the declaration of a
variable may result in a compilation error. This is different from a simple
`fail` which is no coredump on a semantically valid program. To capture these
different conditions, we define new return values for the predicate.

There can be four outcomes when an input is executed:
* Success (failure condition reproduced)
* Failed (failure condition not reproduced)
* Invalid (Did not reach failure condition  possibly semantically invalid)
* Timeout (equivalent to Failed)

In [ ]:
from enum import Enum
import copy

class PRes(str, Enum):
    success = 'SUCCESS'
    failed = 'FAILED'
    invalid = 'INVALID'
    timeout = 'TIMEOUT'

We now define our predicate

In [ ]:
import re

def expr_double_paren(inp):
    if re.match(r'.*[(][(].*[)][)].*', inp):
        return PRes.success
    return PRes.failed

Checking

In [ ]:
if __name__ == '__main__':
    print(expr_double_paren('((1))'))
    print(expr_double_paren('(1)'))

Now, let us define an input, and parse it

We now define a grammar

In [ ]:
import src.grammars as grammars

In [ ]:
import src.utils as utils

In [ ]:
if __name__ == '__main__':
    import ipynb.fs.full.x0_3_Parser as parser
    
    my_input = '1+((2*3/4))'
    expr_parser = parser.EarleyParser(grammars.EXPR_GRAMMAR)
    parsed_expr = list(expr_parser.parse_on(my_input, grammars.EXPR_START))[0]
    utils.display_tree(parsed_expr)

The Perses algorithm is as follows: We start the root, and recursively go down
the child nodes. For each node, we check if that node can be replaced by a
subtree with the same nonterminal, and still reproduce the failure, and find
the smallest such tree (length determined by number of leaves).

Since this procedure can result in multiple trees, the tree to work on is
chosen based on a priority queue where the priority is given to the smallest
tree.

The particular node chosen to replace the current node is determined based
first on its number of leaf nodes, and then on its rank in a priority queue,
where the priority is determined by the depth of the subtree from the current
node. That is, a child gets priority over a grand child.
We first have to define a way to address a specific node.

In [ ]:
if __name__ == '__main__':
    t = parsed_expr[1][0][1][2][1][0]
    print(t)
    utils.display_tree(t)

For the path, we simply use a list of numbers indicating the child node. For example, in the above, the path would be [0, 2, 0]
Given a path, get_child() will simply return the node at the path.

In [ ]:
def get_child(tree, path):
    if not path: return tree
    cur, *path = path
    return get_child(tree[1][cur], path)

Using it.

In [ ]:
if __name__ == '__main__':
    te = get_child(parsed_expr, [0, 2, 0])
    utils.display_tree(te)

We also need a way to replace one node with another. This is done by replace_path().

In [ ]:
def replace_path(tree, path, new_node=None):
    if new_node is None: new_node = []
    if not path: return copy.deepcopy(new_node)
    cur, *path = path
    name, children, *rest = tree
    new_children = []
    for i,c in enumerate(children):
        if i == cur:
            nc = replace_path(c, path, new_node)
        else:
            nc = c
        if nc:
            new_children.append(nc)
    return (name, new_children, *rest)

Using it.

In [ ]:
if __name__ == '__main__':
    t = parsed_expr[1][0][1][2][1][0]
    utils.display_tree(t)
    te = replace_path(parsed_expr, [0, 2, 0], [])
    utils.display_tree(te)
    tn = replace_path(parsed_expr, [0, 2, 0], ('x',[]))
    utils.display_tree(tn)

## Priority queue

For Perses reduction, one needs a way to count the number of leaf nodes to
determine the priority of a node. This is done by count_leaves()

In [ ]:
import heapq

def count_leaves(node):
    name, children, *_ = node
    if not children:
        return 1
    return sum(count_leaves(i) for i in children)

In [ ]:
if __name__ == '__main__':
    print(count_leaves(parsed_expr))
    print(count_leaves(te))

We also define a helper that simply counts the internal nodes.

In [ ]:
def count_nodes(node):
    name, children, *_ = node
    if not children:
        return 0
    return sum(count_nodes(i) for i in children) + 1

In [ ]:
if __name__ == '__main__':
    print(count_nodes(parsed_expr))

Next, we need to maintain a priority queue of the [(tree, path)]. The
essential idea is to prioritize the items first by the number of leaves in  the
full tree (that is, the smallest tree that we have currently gets priority),
then next by the number of leaves in the node pointed to by path, and finally,
tie break by the insertion order (ecount).

In [ ]:
ecount = 0

def add_to_pq(tup, q):
    global ecount
    dtree, F_path = tup
    stree = get_child(dtree, F_path)
    n =  count_leaves(dtree)
    m =  count_leaves(stree)
    # heap smallest first
    heapq.heappush(q, (n, m, -ecount, tup))
    ecount += 1

We define another helper function nt_group() that groups all nonterminals that have the same name. These are used to determine the nodes that can be used to replace one node.

In [ ]:
def nt_group(tree, all_nodes=None):
    if all_nodes is None: all_nodes = {}
    name, children, *_ = tree
    if not utils.is_nt(name): return
    all_nodes.setdefault(name, []).append(tree)
    for c in children:
        nt_group(c, all_nodes)
    return all_nodes

Using it

In [ ]:
if __name__ == '__main__':
    gp = nt_group(te)
    for key in gp:
        print(key)
        for node in gp[key]:
            print(utils.tree_to_str(node))

What are the compatible nodes? These are all the nodes that have the same nonterminal name, and is a descendent of the current node. Further, if the nonterminal allows empty node, then this is the first in the list. This is defined by compatible_nodes()

In [ ]:
def compatible_nodes(tree, grammar):
    key, children, *_ = tree
    # Here is the first choice. Do we restrict ourselves to only children of the tree
    # or do we allow all nodes in the original tree? given in all_nodes?
    lst = nt_group(tree)
    node_lst = [(i, n) for i,n in enumerate(lst[key])]

    # insert empty if the grammar allows it as the first element
    if [] in grammar[key]: node_lst.insert(0, (-1, (key, [])))
    return node_lst

Using it.

In [ ]:
if __name__ == '__main__':
    print(get_child(te, [0]))
    print(compatible_nodes(get_child(te, [0]), grammars.EXPR_GRAMMAR))

Some programming languages have tokens which are first level lexical elements. The parser is often defined using the lexer tokens. We do not want to try to reduce tokens further. So we define a way to identify them (we have to keep in mind when we produce grammars). For now, we assume the ANTLR convention of identifying tokens by uppercase letters.

In [ ]:
def is_token(val):
    assert val != '<>'
    assert (val[0], val[-1]) == ('<', '>')
    if val[1].isupper(): return True
    #if val[1] == '_': return val[2].isupper() # token derived.
    return False

## Simplified Perses reduction
We finally define the reduction algorithm. The implementation of Perses is given in reduction(). The essential idea is as follows:

1. We have a priority queue of (tree, path_to_node) structures, where node is a node within the tree.
  * The highest priority is given to the smallest tree.
  * With in the nodes in the same tree, priority is given to nodes with smallest number of leaves
  * In case of tie break, the shallowest subnode gets the highest priority (i.e child has higher priority over grand child, and empty node has the highest priority since it is a peer of the current node).
2. We pick each nodes, and find compatible subnodes that reproduce the failure.
3. Each compatible node and the corresponding tree is put back into the priority queue.
4. If no child nodes were found that could replace the current node, then we add each children with the current tree into the priority queue. (If we had to recurse into the child nodes, then the next tree that will get picked will be a different tree.)

In [ ]:
def hierarchical_reduction(tree, grammar, predicate):
    first_tuple = (tree, [])
    p_q = []
    add_to_pq(first_tuple, p_q)

    ostr = utils.tree_to_str(tree)
    assert predicate(ostr) == PRes.success
    failed_set = {ostr: True}

    min_tree, min_tree_size = tree, count_leaves(tree)
    while p_q:
        # extract the tuple
        _n, _m, _ec, (dtree, F_path) = heapq.heappop(p_q)
        stree = get_child(dtree, F_path)
        skey, schildren = stree
        found = False
        # we now want to replace stree with alternate nodes.
        for i, node in compatible_nodes(stree, grammar):
            # replace with current (copy).
            ctree = replace_path(dtree, F_path, node)
            if ctree is None: continue # same node
            v = utils.tree_to_str(ctree)
            if v in failed_set: continue
            failed_set[v] = predicate(v) # we ignore PRes.invalid results
            if failed_set[v] == PRes.success:
                found = True
                ctree_size = count_leaves(ctree)
                if ctree_size < min_tree_size: min_tree, min_tree_size = ctree, ctree_size

                if v not in failed_set:
                    print(v)
                t = (ctree, F_path)
                assert get_child(ctree, F_path) is not None
                add_to_pq(t, p_q)

        # The CHOICE here is that we explore the children if and only if we fail
        # to find a node that can replace the current
        if found: continue
        if is_token(skey): continue # do not follow children TOKEN optimization
        for i, child in enumerate(schildren):
            if not utils.is_nt(child[0]): continue
            assert get_child(tree=dtree, path=F_path + [i]) is not None
            t = (dtree, F_path + [i])
            add_to_pq(t, p_q)
    return min_tree

Using it

In [ ]:
if __name__ == '__main__':
    er = hierarchical_reduction(parsed_expr, grammars.EXPR_GRAMMAR, expr_double_paren)
    print(utils.tree_to_str(er))

A question for you. Why doesn't this work?

In [ ]:
if __name__ == '__main__':
    parsed_expr = list(expr_parser.parse_on('1+((2*3/4))*5', grammars.EXPR_START))[0]
    reduced_expr_tree = hierarchical_reduction(parsed_expr, grammars.EXPR_GRAMMAR, expr_double_paren)
    print(utils.tree_to_str(reduced_expr_tree))

Here is a hint.

In [ ]:
if __name__ == '__main__':
    utils.display_tree(reduced_expr_tree)

The problem here is that we don't reduce `<factor> * <term>` to `<factor>`. This can be done if there exist rules that contain a subset of tokens of the current rule. However, in the general case, this requires random generation of nonterminals that are not available in the current rule, which can be costly.

# Done

In [ ]:
#%tb